In [1]:
import sentence_transformers


c:\Users\Sujal\PROJECTS\ProjectX\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from collections import Counter

In [3]:
# --- Import necessary libraries ---
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
import os
from collections import defaultdict

# --- Step 1: Define your ROOT directory (IMPORTANT) ---
pdf_directory = "NcertData"

# --- Step 2: Load ALL PDFs recursively using PyMuPDFLoader ---
print("\n📄 [OBSERVE] Loading all PDF files recursively...\n")

pdf_loader = DirectoryLoader(
    path=pdf_directory,
    glob="**/*.pdf",              # ✅ FIXED: recursive glob
    loader_cls=PyPDFLoader,
    show_progress=True
)

pdf_docs = pdf_loader.load()

print(f"✅ Loaded {len(pdf_docs)} PDF pages from '{pdf_directory}'\n")

# --- Step 3: Normalize paths + group pages by PDF (debug only) ---
pdf_files = defaultdict(list)

for doc in pdf_docs:
    # Normalize Windows paths
    src = doc.metadata.get("source", "Unknown").replace("\\", "/")
    pdf_files[src].append(doc)

# --- Step 4: Preview loaded PDFs (sanity check) ---
print("📘 --- PDF File Previews (Debug) ---\n")

for file_path, docs in list(pdf_files.items())[:5]:  # preview first 5 PDFs
    file_name = os.path.basename(file_path)
    print(f"📄 File: {file_name}")
    print(f"   Pages loaded: {len(docs)}")

    preview_text = docs[0].page_content.strip().replace("\n", " ")
    preview_text = preview_text[:400] + ("..." if len(preview_text) > 400 else "")
    print(f"   📝 Preview: {preview_text}")
    print("-" * 100)


📄 [OBSERVE] Loading all PDF files recursively...



100%|██████████| 865/865 [37:54<00:00,  2.63s/it]  

✅ Loaded 15753 PDF pages from 'NcertData'

📘 --- PDF File Previews (Debug) ---

📄 File: chapter (1).pdf
   Pages loaded: 26
   📝 Preview: Nazism and the Rise of Hitler 49 In the spring of 1945, a little eleven-year-old German boy called Helmuth was lying in bed when he overheard his parents discussing something in serious tones. His father, a prominent physician, deliberated with his wife whether the time had come to kill the entire family, or if he should commit suicide alone. His father spoke about his fear of revenge, saying, ‘No...
----------------------------------------------------------------------------------------------------
📄 File: chapter (2).pdf
   Pages loaded: 22
   📝 Preview: Forest Society and Colonialism 75 In Section II we will shift our focus to the study of livelihoods and economies. We will look at how the lives of forest dwellers and pastoralists changed in the modern world and how they played a part in shaping these changes. All too often in looking at the emerg

In [4]:
def enrich_metadata(doc: Document) -> Document:
    path = doc.metadata["source"].replace("\\", "/")
    parts = path.split("/")

    class_name = subject = language = None

    for i, part in enumerate(parts):
        if part.lower().startswith("class_"):
            class_name = part.replace("Class_", "")
            subject = parts[i + 1]
            language = parts[i + 2]
            break

    doc.metadata.update({
        "class": class_name,
        "subject": subject,
        "language": language,
        "board": "NCERT",
        "content_type": "textbook",
        "source_path": path,
        "page": doc.metadata.get("page", None)
    })

    return doc

In [5]:
enriched_docs = [enrich_metadata(doc) for doc in pdf_docs]
print("✅ Metadata enriched")

✅ Metadata enriched


In [6]:
# Inspect first 5 raw documents
for i, doc in enumerate(pdf_docs[:5], start=1):
    print(f"\n--- RAW Document {i} ---")
    for k, v in doc.metadata.items():
        print(f"{k}: {v}")


--- RAW Document 1 ---
producer: GPL Ghostscript 8.15
creator: PageMaker 7.0
creationdate: 2017-12-08T15:34:40+00:00
author: NCERT
moddate: 2025-04-04T09:50:13+05:30
title: Chapter-3.pmd
source: NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
total_pages: 26
page: 0
page_label: 1
class: None
subject: None
language: None
board: NCERT
content_type: textbook
source_path: NcertData/Class9th/SocialScience/SocialSciencePart4/English/chapter (1).pdf

--- RAW Document 2 ---
producer: GPL Ghostscript 8.15
creator: PageMaker 7.0
creationdate: 2017-12-08T15:34:40+00:00
author: NCERT
moddate: 2025-04-04T09:50:13+05:30
title: Chapter-3.pmd
source: NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
total_pages: 26
page: 1
page_label: 2
class: None
subject: None
language: None
board: NCERT
content_type: textbook
source_path: NcertData/Class9th/SocialScience/SocialSciencePart4/English/chapter (1).pdf

--- RAW Document 3 ---
producer: GPL Ghostscrip

In [7]:
import os
import re
from langchain_core.documents import Document

# Optional language mapping (can be extended)
LANGUAGE_MAP = {
    "english": "en",
    "hindi": "hi",
    "marathi": "mr",
    "malayalam": "ml",
    "urdu": "ur",
    "tamil": "ta",
    "kannada": "kn",
    "telugu": "te"
}

def enrich_metadata(doc: Document) -> Document:
    path = os.path.normpath(doc.metadata["source"])
    parts = path.split(os.sep)

    class_name = None
    subject = None
    language = None

    for i, part in enumerate(parts):
        part_lower = part.lower()

        # ---- Class extraction ----
        if part_lower.startswith("class"):
            match = re.search(r"\d+", part_lower)
            if match:
                class_name = match.group()

            # ---- Subject folder ----
            if i + 1 < len(parts):
                subject = parts[i + 1]

            # ---- Language folder (CASE-NORMALIZED) ----
            if i + 2 < len(parts):
                lang_folder = parts[i + 2].lower()   # 🔑 KEY FIX
                language = LANGUAGE_MAP.get(lang_folder)

            break

    doc.metadata.update({
        "class": class_name,
        "subject": subject,
        "language": language,
        "board": "NCERT",
        "content_type": "textbook",
        "source_path": path,
        "page": doc.metadata.get("page")
    })

    return doc

In [8]:
enriched_docs = [enrich_metadata(doc) for doc in pdf_docs]
print("✅ Metadata re-extracted from source_path")

✅ Metadata re-extracted from source_path


In [9]:
for d in enriched_docs[:]:
    print(
        d.metadata["class"],
        d.metadata["subject"],
        d.metadata["language"],
        d.metadata["source_path"]
    )

9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
9 SocialScience None NcertData

In [10]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [11]:
def is_image_heavy(doc, min_chars=100):
    return len(doc.page_content.strip()) < min_chars

In [12]:
text_docs = []
image_docs = []

for doc in pdf_docs:
    if is_image_heavy(doc):
        image_docs.append(doc)
    else:
        text_docs.append(doc)

print("Text pages:", len(text_docs))
print("Image-heavy pages:", len(image_docs))

Text pages: 13800
Image-heavy pages: 1953


In [13]:
from pdf2image import convert_from_path
import pytesseract

def ocr_page(pdf_path, page_number):
    images = convert_from_path(
        pdf_path,
        first_page=page_number + 1,
        last_page=page_number + 1,
        dpi=300
    )
    text = pytesseract.image_to_string(
        images[0],
        lang="eng+hin"   # add more langs if needed
    )
    return text.strip()

In [14]:
ocr_docs = []

for doc in image_docs:
    pdf_path = doc.metadata["source"]
    page_no = doc.metadata["page"]

    try:
        ocr_text = ocr_page(pdf_path, page_no)
    except Exception as e:
        continue

    if len(ocr_text) > 30:
        doc.page_content = ocr_text
        doc.metadata["page_type"] = "ocr"
        ocr_docs.append(doc)

In [15]:
final_docs = text_docs + ocr_docs

print("Final usable pages:", len(final_docs))

Final usable pages: 13800


In [16]:
def clean_base_text(text: str) -> str:
    # Normalize newlines
    text = re.sub(r"\n{2,}", "\n", text)

    # Normalize spaces
    text = re.sub(r"[ \t]{2,}", " ", text)

    # Remove common bullet artifacts
    text = re.sub(r"[•■▪◆►]", " ", text)

    # Remove isolated page numbers
    text = re.sub(r"\n?\s*Page\s+\d+\s*\n?", "\n", text, flags=re.IGNORECASE)

    return text.strip()

In [17]:
def clean_ocr_text(text: str) -> str:
    # Fix broken hyphenation across lines
    text = re.sub(r"-\n", "", text)

    # Remove excessive line breaks
    text = re.sub(r"\n{3,}", "\n\n", text)

    # Remove OCR garbage characters
    text = re.sub(r"[|_~]", "", text)

    return clean_base_text(text)

In [18]:
def clean_document(doc: Document) -> Document:
    page_type = doc.metadata.get("page_type", "text")

    if page_type == "ocr":
        cleaned_text = clean_ocr_text(doc.page_content)
    else:
        cleaned_text = clean_base_text(doc.page_content)

    return Document(
        page_content=cleaned_text,
        metadata=doc.metadata
    )

In [19]:
cleaned_docs = [clean_document(doc) for doc in final_docs]

print(f"✅ Cleaned documents: {len(cleaned_docs)}")

✅ Cleaned documents: 13800


In [20]:
for d in cleaned_docs:
    if d.metadata.get("page_type") != "ocr":
        print(d.page_content[:300])
        break

Nazism and the Rise of Hitler
49
In the spring of 1945, a little eleven-year-old German boy called
Helmuth was lying in bed when he overheard his parents discussing
something in serious tones. His father, a prominent physician,
deliberated with his wife whether the time had come to kill the entire
f


In [21]:
for d in cleaned_docs:
    if d.metadata.get("page_type") == "ocr":
        print(d.page_content[:300])
        break

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,          # ideal for theory + definitions
    chunk_overlap=90,       # preserves continuity
    separators=[
        "\n\n",              # paragraphs
        "\n",                # lines
        ". ",                # sentences
        " "                  # fallback
    ]
)

In [24]:
chunked_docs = text_splitter.split_documents(cleaned_docs)

print(f"✅ Total chunks created: {len(chunked_docs)}")

✅ Total chunks created: 63754


In [25]:
for d in chunked_docs[:]:
    print(
        d.metadata.get("class"),
        d.metadata.get("subject"),
        d.metadata.get("language"),
        d.metadata.get("page"),
        d.metadata.get("page_type")
    )

9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 0 None
9 SocialScience None 1 None
9 SocialScience None 1 None
9 SocialScience None 1 None
9 SocialScience None 1 None
9 SocialScience None 2 None
9 SocialScience None 2 None
9 SocialScience None 2 None
9 SocialScience None 2 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 3 None
9 SocialScience None 4 None
9 SocialScience None 4 None
9 SocialScience None 4 None
9 SocialScience None 4 None
9 SocialScience None 4 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None 5 None
9 SocialScience None

In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [27]:
# LangChain core packages
import langchain
import langchain_core
import langchain_community
import langchain_text_splitters

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Vector DB
from langchain_community.vectorstores import Chroma

print("✅ All modules imported successfully")

✅ All modules imported successfully


In [28]:
import sentence_transformers
print(sentence_transformers.__version__)

5.2.0


In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings1 = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True}
)

vec = embeddings1.embed_query("query: test")
print(len(vec))

C:\Users\Sujal\AppData\Local\Temp\ipykernel_29896\1457127348.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings1 = HuggingFaceEmbeddings(


1024


In [30]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={
        "normalize_embeddings": True
    }
)

print("✅ multilingual-e5-large embeddings loaded")


✅ multilingual-e5-large embeddings loaded


In [31]:
from langchain_core.documents import Document

e5_docs = [
    Document(
        page_content="passage: " + d.page_content,
        metadata=d.metadata
    )
    for d in chunked_docs
]

print("✅ E5 formatting applied to documents")

✅ E5 formatting applied to documents


In [32]:
print(e5_docs[0].page_content[:50])
# should start with "passage:"

passage: Nazism and the Rise of Hitler
49
In the s


In [33]:
print(f"Documents ready for embedding: {len(e5_docs)}")

Documents ready for embedding: 63754


In [34]:
from langchain_community.vectorstores import Chroma

PERSIST_DIR = "./chroma_ncert_db"

vectorstore = Chroma.from_documents(
    documents=e5_docs,
    embedding=embeddings,
    persist_directory=PERSIST_DIR,
    collection_name="ncert_multilingual"
)

vectorstore.persist()

print(f"✅ Embedded and stored {len(e5_docs)} chunks")


✅ Embedded and stored 63754 chunks


C:\Users\Sujal\AppData\Local\Temp\ipykernel_29896\2007844869.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [35]:
vectorstore._collection.count()

63754

In [36]:
sample = vectorstore._collection.get(
    limit=3,
    include=["documents", "metadatas", "embeddings"]
)

for i in range(len(sample["documents"])):
    print(f"\n--- VECTOR {i+1} ---")
    print("Text (first 300 chars):")
    print(sample["documents"][i][:300])
    print("\nMetadata:")
    print(sample["metadatas"][i])
    print("\nEmbedding dimension:", len(sample["embeddings"][i]))


--- VECTOR 1 ---
Text (first 300 chars):
passage: Nazism and the Rise of Hitler
49
In the spring of 1945, a little eleven-year-old German boy called
Helmuth was lying in bed when he overheard his parents discussing
something in serious tones. His father, a prominent physician,
deliberated with his wife whether the time had come to kill the

Metadata:
{'class': '9', 'creationdate': '2017-12-08T15:34:40+00:00', 'author': 'NCERT', 'total_pages': 26, 'page': 0, 'moddate': '2025-04-04T09:50:13+05:30', 'content_type': 'textbook', 'source': 'NcertData\\Class9th\\SocialScience\\SocialSciencePart4\\English\\chapter (1).pdf', 'page_label': '1', 'creator': 'PageMaker 7.0', 'source_path': 'NcertData\\Class9th\\SocialScience\\SocialSciencePart4\\English\\chapter (1).pdf', 'subject': 'SocialScience', 'producer': 'GPL Ghostscript 8.15', 'title': 'Chapter-3.pmd', 'board': 'NCERT'}

Embedding dimension: 1024

--- VECTOR 2 ---
Text (first 300 chars):
passage: his fear of revenge, saying, ‘Now the Allie

In [37]:
sample = vectorstore._collection.get(
    limit=1,
    include=["embeddings"]
)

print(type(sample["embeddings"]))
print(len(sample["embeddings"][0]))

<class 'numpy.ndarray'>
1024


In [38]:
sample = vectorstore._collection.get(
    limit=10,
    include=["documents", "metadatas", "embeddings"]
)

for i in range(len(sample["documents"])):
    print(f"\n========== VECTOR {i+1} ==========")

    # Show text (trimmed)
    print("📄 Stored Text (first 300 chars):")
    print(sample["documents"][i][:300])

    # Show metadata
    print("\n🧾 Metadata:")
    for k, v in sample["metadatas"][i].items():
        print(f"  {k}: {v}")

    # Show embedding info
    print("\n📐 Embedding Dimension:")
    print(len(sample["embeddings"][i]))


========== VECTOR 1 ==========
📄 Stored Text (first 300 chars):
passage: Nazism and the Rise of Hitler
49
In the spring of 1945, a little eleven-year-old German boy called
Helmuth was lying in bed when he overheard his parents discussing
something in serious tones. His father, a prominent physician,
deliberated with his wife whether the time had come to kill the

🧾 Metadata:
  total_pages: 26
  source: NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
  subject: SocialScience
  title: Chapter-3.pmd
  source_path: NcertData\Class9th\SocialScience\SocialSciencePart4\English\chapter (1).pdf
  page: 0
  moddate: 2025-04-04T09:50:13+05:30
  page_label: 1
  content_type: textbook
  board: NCERT
  class: 9
  creationdate: 2017-12-08T15:34:40+00:00
  author: NCERT
  producer: GPL Ghostscript 8.15
  creator: PageMaker 7.0

📐 Embedding Dimension:
1024

========== VECTOR 2 ==========
📄 Stored Text (first 300 chars):
passage: his fear of revenge, saying, ‘Now the Allies 

In [39]:
emb = vectorstore._collection.get(limit=1, include=["embeddings"])
print(emb["embeddings"][0][:10])   # first 10 numbers of the vector

[ 0.02565398 -0.00828565 -0.03042093 -0.0193582   0.02259064 -0.02144116
  0.01508121  0.07221963  0.05898706 -0.0140533 ]


In [4]:
# =========================
# Voice Input (Windows-safe): PyAudio
# =========================

import pyaudio
import wave

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 6
OUTPUT_FILE = "query.wav"

audio = pyaudio.PyAudio()

stream = audio.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK
)

print("🎙️ Recording... Speak now")

frames = []
for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("✅ Recording finished")

stream.stop_stream()
stream.close()
audio.terminate()

with wave.open(OUTPUT_FILE, "wb") as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))

print("Saved as query.wav")

🎙️ Recording... Speak now
✅ Recording finished
Saved as query.wav


In [26]:
# =========================
# Voice → Text (Whisper)
# =========================
import whisper

# Load once per session
whisper_model = whisper.load_model("base")  # good for Malayalam

result = whisper_model.transcribe(
    "query.wav",
    fp16=False,
    language=None   # auto-detect
)

voice_query = result["text"].strip()
print("🗣️ Transcribed Query:")
print(voice_query)

c:\Users\Sujal\PROJECTS\ProjectX\.venv\Lib\site-packages\whisper\__init__.py:69: UserWarning: C:\Users\Sujal\.cache\whisper\base.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|███████████████████████████████████████| 139M/139M [02:07<00:00, 1.14MiB/s]


🗣️ Transcribed Query:
to draw the strength is the longest


In [27]:
# =========================
# Save query for Streamlit
# =========================
import json

with open("voice_query.json", "w", encoding="utf-8") as f:
    json.dump({"query": voice_query}, f, ensure_ascii=False)

print("✅ voice_query.json saved")

✅ voice_query.json saved
